In [1]:
import os

In [2]:
%pwd

'f:\\DS\\TextSummarizer\\Text-Summarizer\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'f:\\DS\\TextSummarizer\\Text-Summarizer'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_dir: Path
    tokenizer_name: Path

In [9]:
from textSummarizer.constants import *
from textSummarizer.utils.common_utils import read_yaml, create_directories

In [17]:
class ConfigurationManager:

    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def fetch_datatransformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_dir=config.data_dir,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config


In [11]:
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [20]:
class DataTransformation:

    def __init__(self, config=DataTransformationConfig) -> None:
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_example_to_features(self, example_batch):

        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):

        dataset_samsum = load_from_disk(self.config.data_dir)
        dataset_samsum_pt = dataset_samsum.map(self.convert_example_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [21]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.fetch_datatransformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e

[2024-03-28 17:54:06,568: INFO: common_utils: yaml file: config\config.yaml loaded successfully]
[2024-03-28 17:54:06,572: INFO: common_utils: yaml file: params.yaml loaded successfully]
[2024-03-28 17:54:06,574: INFO: common_utils: Created directory at: artifacts]
[2024-03-28 17:54:06,575: INFO: common_utils: Created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\Hi\.conda\envs\textS\lib\site-packages\transformers\tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 68186.51 examples/s]
